In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['mnist_train.csv', 'mnist_test.csv']


In [80]:
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [81]:
train = pd.read_csv("../input/mnist_train.csv")
test = pd.read_csv("../input/mnist_test.csv")

In [82]:
print(train.shape)
print(test.shape)
print(train.columns)
print(train[:2])
print(test[:2])
print(train[:10]["label"])
print(test[:10]["label"])

(60000, 785)
(10000, 785)
Index(['label', '1x1', '1x2', '1x3', '1x4', '1x5', '1x6', '1x7', '1x8', '1x9',
       ...
       '28x19', '28x20', '28x21', '28x22', '28x23', '28x24', '28x25', '28x26',
       '28x27', '28x28'],
      dtype='object', length=785)
   label  1x1  1x2  1x3  1x4  ...    28x24  28x25  28x26  28x27  28x28
0      5    0    0    0    0  ...        0      0      0      0      0
1      0    0    0    0    0  ...        0      0      0      0      0

[2 rows x 785 columns]
   label  1x1  1x2  1x3  1x4  ...    28x24  28x25  28x26  28x27  28x28
0      7    0    0    0    0  ...        0      0      0      0      0
1      2    0    0    0    0  ...        0      0      0      0      0

[2 rows x 785 columns]
0    5
1    0
2    4
3    1
4    9
5    2
6    1
7    3
8    1
9    4
Name: label, dtype: int64
0    7
1    2
2    1
3    0
4    4
5    1
6    4
7    9
8    5
9    9
Name: label, dtype: int64


In [83]:
train = np.array(train)
test = np.array(test)

In [84]:
print(train[10:15])
print(test[10:15])

[[3 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [85]:
train_x = train[:,1:]
train_y = pd.get_dummies(train[:,0])
test_x = test[:,1:]
test_y = pd.get_dummies(test[:,0])

In [86]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(train_x[10:12])
print(train_y[10:12])
print(test_x[10:12])
print(test_y[10:12])

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
    0  1  2  3  4  5  6  7  8  9
10  0  0  0  1  0  0  0  0  0  0
11  0  0  0  0  0  1  0  0  0  0
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
    0  1  2  3  4  5  6  7  8  9
10  1  0  0  0  0  0  0  0  0  0
11  0  0  0  0  0  0  1  0  0  0


#### We will treat each image as a sequence of 28 rows of 28 pixels each (since each MNIST image is 28 × 28 pixels). We will use cells of 150 recurrent neurons, plus a fully connected layer containing 10 neurons (one per class) connected to the output of the last time step, followed by a softmax layer

In [87]:
#NETWORK PARAMETERS
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [88]:
train_x = train_x.reshape(-1,n_steps,n_inputs)
test_x = test_x.reshape(-1,n_steps,n_inputs)

In [89]:
print(train_x.shape)
print(test_x.shape)
print(train_x[0:1])
print(test_x[0:1])

(60000, 28, 28)
(10000, 28, 28)
[[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126
   136 175  26 166 255 247 127   0   0   0   0]
  [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253
   253 225 172 253 242 195  64   0   0   0   0]
  [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253
   251  93  82  82  56  39   0   0   0   0   0]
  [  0   0   0  

In [90]:
train_X , train_y = shuffle(train_x , train_y)
test_X , test_y = shuffle(test_x , test_y)

In [91]:
tf.reset_default_graph()

In [92]:
# Training Parameters
learning_rate = 0.001
training_iters = 100
batch_size = 150
display_step = 200

In [93]:
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_outputs])

In [94]:
def RNN(X):
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs , states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
    out = tf.layers.dense(states, n_outputs)
    out = tf.nn.softmax(out)
    return out
    

In [95]:
prediction = RNN(X)

In [96]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [97]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [99]:
sess = tf.Session()
sess.run(init) 

for i in range(training_iters):
    for batch in range(len(train_X)//batch_size):
        batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    

        sess.run(train_op, feed_dict={X: batch_x, y: batch_y})
            # Calculate batch loss and accuracy
        loss = sess.run([loss_op], feed_dict={X: batch_x, y: batch_y})
    
    predTest = sess.run(prediction , feed_dict={X:test_X})

    p = np.argmax(predTest,1)
    t = np.argmax(np.array(test_y),1)

    acc = accuracy_score(p,t)
    print("Iter "+str(i)+" Out of",training_iters , " Loss= ",loss, "acc=",acc )
            
#     acc = sess.run([accuracy], feed_dict={X: batch_x, y: batch_y})
        
#     print("Step " + str(i) + ",        Batch Loss= ",loss, ",       Training Accuracy= ",acc)
    
print("Optimization Finished!")

Iter 0 Out of 100  Loss=  [1.8855386] acc= 0.5067
Iter 1 Out of 100  Loss=  [1.8827469] acc= 0.5075
Iter 2 Out of 100  Loss=  [1.8734704] acc= 0.5131
Iter 3 Out of 100  Loss=  [1.8806031] acc= 0.5146
Iter 4 Out of 100  Loss=  [1.8724397] acc= 0.5199
Iter 5 Out of 100  Loss=  [1.8630449] acc= 0.5186
Iter 6 Out of 100  Loss=  [1.8721691] acc= 0.5211
Iter 7 Out of 100  Loss=  [1.8643327] acc= 0.5175
Iter 8 Out of 100  Loss=  [1.8711295] acc= 0.5181
Iter 9 Out of 100  Loss=  [1.8555135] acc= 0.5238
Iter 10 Out of 100  Loss=  [1.8639575] acc= 0.5242
Iter 11 Out of 100  Loss=  [1.8654317] acc= 0.5234
Iter 12 Out of 100  Loss=  [1.8529726] acc= 0.5316
Iter 13 Out of 100  Loss=  [1.8765159] acc= 0.5278
Iter 14 Out of 100  Loss=  [1.8703305] acc= 0.5263
Iter 15 Out of 100  Loss=  [1.8799516] acc= 0.5287
Iter 16 Out of 100  Loss=  [1.8528666] acc= 0.5309
Iter 17 Out of 100  Loss=  [1.8494303] acc= 0.5318
Iter 18 Out of 100  Loss=  [1.8484768] acc= 0.5335
Iter 19 Out of 100  Loss=  [1.861479] acc

In [100]:
while(True):
    r = np.random.randint(9000)
    test_img = np.reshape(test_X[r], (28,28))
    plt.imshow(test_img, cmap="gray")
    test_pred = sess.run(prediction, feed_dict = {X:[test_X[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred))
    plt.show()
    
    if input("Enter n to exit")=='n':
        break
clear_output();

In [101]:
wrong = test_X[t!=p]
wrong.shape

(4498, 28, 28)

In [102]:
a,b,c = wrong.shape

In [104]:
while(True):
    r=np.random.randint(a)
    plt.imshow(wrong[r].reshape((28,28)),cmap="gray")
    test_pred_1=sess.run(prediction, feed_dict = {X:[wrong[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred_1))
    plt.show()
    
    if input("Enter n to exit")=='n':
        break
clear_output();

In [106]:
p = np.argmax(predTest,1)
print(p)
t = np.argmax(np.array(test_y),1)
print(t)
acc = accuracy_score(p,t)
print(acc*100)

[6 6 1 ... 2 4 8]
[6 6 4 ... 2 8 2]
55.02


In [107]:
print("Saving Weights")
saver = tf.train.Saver()
saver.save(sess,"weights_"+str(i)+"/weights.ckpt")
print("Weights Saved")

Saving Weights
Weights Saved
